# Practice NumPy

Make sure you look at [`Intro_to_NumPy.ipynb`](Intro_to_NumPy.ipynb) first!

In [ ]:
import numpy as np

from utils import vp_from_dt, impedance, rc_series

In [ ]:
test = np.random.random(10000000)

In [ ]:
%timeit rc_series(test)

Note that the log has to be fairly big for the benchmarking to work properly, because otherwise the CPU caches the computation and this skews the results.

Now we can re-write our function using arrays instead of lists. Let's just remind ourselves of what it looked like before:

In [ ]:
import inspect
print(inspect.getsource(rc_series))

## Exercise

- Recall your `impedance()` function. How would you write that for NumPy arrays?
- Rewrite the `vp_from_dt()` function as a NumPy operation. Make sure the function continues to work on lists by casting the input to arrays.
- Rewrite the `rc_series()` function to get rid of the loop. Remember that the math operations work on the entire array at once.
- Time your new RC series function on the random `test` data and compare to the loop version.
- Replace the functions in your `utils.py` file with vectorized versions.

In [ ]:
def vp_from_dt(dt):
    
    # YOUR CODE HERE

In [ ]:
def rc_vector(imp):
    
    # YOUR CODE HERE

In [ ]:
z = np.arange(10)
rc_vector(z)

You should get the same output as you did before:

    array([1.        , 0.33333333, 0.2       , 0.14285714, 0.11111111,
       0.09090909, 0.07692308, 0.06666667, 0.05882353])

In [ ]:
def vp_from_dt(dt):
    return 1e6 / dt

In [ ]:
def rc_vector(z):
    uppers = z[:-1]
    lowers = z[1:]
    return (lowers - uppers) / (uppers + lowers)

In [ ]:
%timeit rc_vector(test)

60+ times faster on my machine!

## Load and use some real data

In [1]:
from welly import Well

w = Well.from_las('https://geocomp.s3.amazonaws.com/data/R-39.las', index='original')

gr = w.data['GR']
dt = w.data['DT4P']
rhob = w.data['RHOB']
dts = w.data['DT4S']

/home/matt/miniconda3/envs/geocomp/lib/python3.9/site-packages/welly/well.py:194: FutureWarning: From v0.5 the default will be 'original', keeping whatever is used in the LAS file. If you want to force conversion to metres, change your code to use `index='m'`.
  warnings.warn(m, FutureWarning)


### Exercise

- Complete the following lines to run the vectorized `rc_series` function on the log data to make an RC series array.
- Plot the array using the cell provided.

In [ ]:
vp = 
vs = 
imp = 

In [ ]:
vp = vp_from_dt(dt)
vs = vp_from_dt(dts)
imp = impedance(rhob, vp)

In [ ]:
import matplotlib.pyplot as plt

depth = dt.basis

plt.figure(figsize=(2, 10))
plt.plot(imp, depth)
plt.show()

## OPTIONAL STOPPING POINT

# Compute a time-to-depth relationship

The time-to-depth relationship taking a start-time for the top of the log `tstart`, and then by adding the cumulative travel time for each depth sample beneath it. 

We obtain the cumulative travel-time by scaling the p-sonic log by the sample interval (6" or 0.1524 m) and by calling `np.cumsum()` on it. 

In [ ]:
tstart = 1.300

In [ ]:
# two-way-time to depth relationship
scaled_dt = 0.1524 * np.nan_to_num(dt) / 1e6
tcum = 2 * np.cumsum(scaled_dt)
tdr = tcum + tstart

Let's look at our time-depth curve

In [ ]:
plt.figure(figsize = (4,5))
plt.plot(tdr, depth)
plt.plot([0, tdr[0]], [0, depth[0]], '--', c='grey')
plt.grid(lw=0.5)
plt.xlim(0, 2.0)
plt.ylim(3500, 0)
plt.ylabel('TVD ss [m]')
plt.xlabel('two-way-time [s]')

It looks pretty close to being linear over this depth range, but it isn't quite linear!

## Convert the logs to time and compute RC

In [ ]:
delta_t = 0.001 # sample rate of the data
mint = 0.0  # the shallowest travel time we care about (seismic reference datum)
maxt = 2.5  # the deepest travel time we care about
t = np.arange(mint, maxt, delta_t)

gr_t = np.interp(x=t, xp=tdr, fp=gr)
imp_t = np.interp(x=t, xp=tdr, fp=imp)

In [ ]:
rc = rc_vector(imp_t)

## Convolution

Now we'll use the RC to compute a synthetic.

In [ ]:
from bruges.filters import ricker

wavelet, tw = ricker(0.128, delta_t, 25.0, return_t=True)

plt.plot(wavelet)

In [ ]:
syn = np.convolve(rc, wavelet, mode='same')

In [ ]:
t_, syn_ = t[1500:2000], syn[1500:2000]

plt.figure(figsize=(15, 2))
plt.plot(t_, syn_)
plt.fill_between(t_, 0, syn_, where=syn_>0)
plt.show()

## OPTIONAL STOPPING POINT

## Compute offset synthetic

In [ ]:
vp_t = np.interp(x=t, xp=tdr, fp=vp)
vs_t = np.interp(x=t, xp=tdr, fp=vs)
rhob_t = np.interp(x=t, xp=tdr, fp=rhob)

In [ ]:
from bruges.reflection import reflectivity

theta = np.arange(46)

vs_t[vs_t < 0] = vp_t[vs_t < 0] / 2

rc_theta = reflectivity(vp_t, vs_t, rhob_t, theta).T

In [ ]:
plt.imshow(rc_theta.real, aspect='auto')

In [ ]:
def convolve(tr, w):
    return np.convolve(tr, w, mode='same')

s = np.apply_along_axis(convolve, 0, rc_theta.real, wavelet)

In [ ]:
plt.figure(figsize=(6, 10))
plt.imshow(s, cmap="RdBu", aspect='auto', clim=[-0.25, 0.25], extent=[theta[0], theta[-1], t[-1], t[0]])
plt.ylim(2.0, 1.2)
plt.colorbar()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(7, 10),
                        ncols=3,
                        sharey=True,
                        gridspec_kw=dict(width_ratios=[1.5, 1, 1 ]),
                       )

# Plot synthetic gather.
ax = axs[0]
ax.imshow(s, cmap="seismic", aspect='auto', clim=(-0.35,0.35), extent=[0,60, t[-1], t[0]])
gain = 10
for i, tr in enumerate(s.T):
    if i % 2 == 1:
        axs[0].plot(gain*(tr)+i, t[:-1], 'k', alpha=0.5)
ax.set_xlim(0,45)
ax.set_ylim(2.0, 1.2)
ax.set_xlabel('two-way-time (s)')
ax.set_title('incidence angle ($\degree$)')

# Plot impedance log.
ax = axs[1]
ax.plot(imp_t, t,  lw=1.0)
ax.set_xlim(np.percentile(imp,5)*0.8, np.percentile(imp,95)*1.2)
ax.grid(c='k', alpha=0.25)
ax.set_yticks([])
ax.set_title('impedance')

# Plot colour-filled GR.
ax = axs[2]
ax.plot(gr_t, t,  c='k',lw=1.0)
ax.fill_betweenx(t, gr_t, 0, color='lightgrey')
ax.fill_betweenx(t, gr_t, 100, color='khaki')
ax.grid(c='k', alpha=0.25)
ax.set_xlim(20,100)
ax.set_yticks([])
ax.set_xticks([25,50,75,100])
ax.grid(lw=0.5)
ax.set_title('gamma ray (API)')

plt.show()

## OPTIONAL STOPPING POINT

----

## Vsh

_V_<sub>sh</sub> or _V_<sub>shale</sub> is the volume of shale in a given volume of rock. Often synonymous with _V_<sub>clay</sub>, though strictly speaking this should be measured at a different scale: _V_<sub>clay</sub> pertains to a rock, whereas _V_<sub>sh</sub> pertains to an interval of strata. 

It is possible to calculate _V_<sub>sh</sub> from spectral gamma-ray CGR curve data (usually where GR comes from):

$$x = \frac{\mathsf{CGR}_\mathrm{zone} - \mathsf{CGR}_\mathrm{clean}}{\mathsf{CGR}_\mathrm{shale} - \mathsf{CGR}_\mathrm{clean}}$$

In many circumstances, _x_ can be used as _V_<sub>sh</sub>. Alternatively, one of the following corrections can be optionally applied:

$V_\mathrm{sh} = \frac{0.5x}{1.5-x}$

$V_\mathrm{sh} = 1.7 - \sqrt{3.38 - (x + 0.7)2}$

### Exercise

Implement the Vsh equation.

- Your function should work on scalars and on arrays or other sequences.
- The function should never return a number outside the closed interaval [0, 1].
- Write a docstring and tests for your function.
- Apply your function to the GR log from the well `w`

In [ ]:
def vshale(cgr, clean, shale):

    # Your code here!
    
    return vsh

In [ ]:
cgr = [40, 50, 80, 100, 120, 80, np.nan, 10]
vshale(cgr, clean=40, shale=100)

This should yield:

    array([ 0.    ,  0.0625,  0.4   ,  1.    ,  1.    ,  0.4   ,     nan,  0.    ])

In [ ]:
def vshale(cgr, clean, shale):
    """
    Compute VSH for arrays or single values.
    """
    cgr = np.atleast_1d(cgr)
    
    # If we don't like the warnings, we can temporarily
    # replace the nans.
    nans = np.isnan(cgr)
    cgr[np.isnan(cgr)] = 0

    x = (cgr - clean) / (shale - clean)
    vsh = 0.5 * x / (1.5 - x)
    
    # Make sure we're in the interval [0, 1]
    vsh[vsh > 1] = 1
    vsh[vsh < 0] = 0

    # Replace the nans.
    vsh[nans] = np.nan
    
    return np.squeeze(vsh)

In [ ]:
vshale(cgr, clean=40, shale=100)

In [ ]:
vshale(45, 40, 100)

In [2]:
w = Well.from_las('https://geocomp.s3.amazonaws.com/data/R-39.las', index='original')

vsh = vshale(w.data['GR'], 40, 100)
depth = w.data['GR'].basis

plt.figure(figsize=(2, 8))
plt.plot(vsh[:200], depth[:200])
plt.title('Vshale')
plt.show()

NameError: name 'vshale' is not defined

----

## Confining pressure

The confining pressure as a function of depth is given by:

$$p_{c}\left ( z \right )=p_{0}+g\int_{0}^{z}\rho (z)dz$$

where $\rho_{z}$ is the density of the overlying rock at depth z, and g is the acceleration due to gravity, $9.81 m/s^{2}$, $p_{0}$ is the datum pressure, the pressure at the surface. Hint: calculate the $p_{0}$ to using by passing a constant density down to the top of the of shallowest log measurement. $$p_{0}=\rho_{0}gz_{0}$$

### Exercise

- Write a function implementing the equation shown above.
- Your function should work on scalars and on arrays or other sequences.
- The function should take a optional keyword argument to return pressure in Pa (Pascals) or MPa (MegaPascals).
- Write a docstring and tests for your function.
- Apply your function to the `RHOB` log from the well `w`, and make a plot

In [ ]:
def confining_pressure(rhob, rho0=2200.0, z0=0):

    # Your code here!
    
    return pconf

In [ ]:
def confining_pressure(rhob, rho0=2200.0, z0=0):

    # Your code here!
    
    return pconf

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2018</p>
</div>